In [19]:
from splinter import Browser
import re
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

In [20]:
#Set up Splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/terencelin/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [23]:
nvidia_gpus = ['https://www.zoolert.com/computers/videocards/nvidia/rtx-3070/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3070-ti/instockhistory.php',
              'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080-ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3090/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060/instockhistory.php'
              ]
descriptions = []
time = []
for references in nvidia_gpus:
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    browser.find_by_name('history_length').first.select('200')
    data = gpu_soup.find_all('tr',role = "row")

    for value,link in enumerate(data):
        descriptions.append(link.get_text(separator = "\n"))
        time.append(descriptions[value].strip().split("\n")[-1])

print(len(time))
    

1161


In [24]:
for ref in descriptions:
    print(ref.strip().split("\n")[-1].strip())

Total Time
1m
5h 36m
1m
4m
19h 48m
4m
1m
1m
4m
1m
1m
1m
1m
13d 1h 44m
4m
4d 40m
10h 56m
14h 12m
1d 2h 24m
1d 23h 0m
7h 56m
1m
1d 7h 36m
1m
24m
4m
4m
8m
1h 12m
1h 40m
5h 8m
2h 52m
1m
1h 0m
8m
8m
8m
3h 36m
4m
4m
1m
1m
4m
1m
1m
1m
1m
1m
4m
1m
4m
1m
1m
4m
1m
4m
1m
4m
4m
1m
28m
4m
1m
4m
4m
4m
1m
4m
8m
8m
1m
1m
1m
4m
4m
1m
28m
28m
3h 32m
52m
1m
32m
8m
1m
1m
1m
28m
23h 24m
1d 7h 53m
1m
1m
24m
1m
1m
1m
1m
1m
8m
24m
1m
1m
1m
3m
4m
3m
12m
1m
3m
24m
1m
1m
3m
1m
1m
1m
27m
27m
27m
24m
3m
24m
1m
1m
1m
1m
24m
3m
27m
1m
2m
2m
12m
2m
4m
2m
5m
2m
4m
1m
28m
4m
12m
8m
9m
2m
3m
28m
28m
28m
28m
1h 58m
12m
28m
20h 9m
4m
4m
1m
29m
4m
51m
2m
3m
14m
3m
2m
1m
3m
2m
32m
1m
1m
2m
1m
1m
1m
1m
3m
1m
1m
2m
1m
4m
2m
1m
1m
1m
1m
1m
1m
2m
2m
1m
1m
2m
2m
2m
1m
1m
2m
1m
Total Time
3h 7m
2m
1h 37m
1m
23m
1h 2m
1h 0m
48m
1h 7m
15m
15m
20m
1m
1m
48m
1m
7m
27m
1h 7m
24m
1h 27m
9m
10m
1h 35m
7m
3h 42m
35m
12m
4m
1m
20m
11m
2h 7m
14m
11m
16m
20m
1m
4m
2h 1m
1m
1m
1m
1h 22m
12m
2m
2m
19m
10m
27m
1m
14m
14m
7m
7m
7m
1m
9m
1m
1m
7

In [25]:
time.remove('Total Time')

In [26]:

stores = [ref.strip().split("\n")[0] for ref in descriptions if ref.strip().split("\n")[0] is not 'Store']
stores.remove('Store')
print(stores)

['Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon CA', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazon', 'Amazo